In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [2]:
df = pd.read_csv("sim_data_full.csv")

In [3]:
from compute_costs import *

In [4]:
compute_costs(df)

C:\Users\joshsu\OneDrive - Bill & Melinda Gates Foundation\Code\malaria-sac-ipt\analysis\230503\compute_costs.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_school["cost_IPTsc_drugs"] = df_school["iptsc_drugs_used_school"] * df_school["drug_type"].map(cost_dict)
C:\Users\joshsu\OneDrive - Bill & Melinda Gates Foundation\Code\malaria-sac-ipt\analysis\230503\compute_costs.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_school["cost_IPTsc_consumables"] = df_school["cost_IPTsc_RDTs"] + df_schoo

In [5]:
df_pfpr = df[df["baseline_transmission_metric"]=="pfpr"].reset_index(drop=True)
df_pfpr = df_pfpr[df_pfpr["transmission_level"] > 0.01].reset_index(drop=True)

In [6]:
df_plot = df_pfpr[np.logical_and.reduce([
    df_pfpr["archetype"]=="Sahel",
    df_pfpr["transmission_level"]==0.2,
    np.in1d(df_pfpr["scenario_number"], [6,7,8,9,30,31,32,33])])]

In [7]:
df_itn = df_pfpr[np.logical_and.reduce([
    df_pfpr["archetype"]=="Sahel",
    df_pfpr["transmission_level"]==0.2,
    np.in1d(df_pfpr["scenario_number"], [5])])]

In [8]:
school_marker = "o"
smc_marker = "s"

marker_dict = {i: school_marker for i in [6,7,8,9,10]}
marker_dict.update({i: smc_marker for i in [30,31,32,33]})

In [9]:
color_dict = {
    6: "#5292C2",
    7: "#5292C2",
    8: "#F48420",
    9: "#F48420",
    5: "black",
    35: "#E9CA43",
    36: "#D61C5B"
}

In [10]:
df_plot[["scenario_number", "cost_IPTsc_overhead_low", "cost_IPTsc_overhead_high", "cost_IPTsc_high", "cost_IPTsc_low", "cost_IPTsc_mean"]]

,scenario_number,cost_IPTsc_overhead_low,cost_IPTsc_overhead_high,cost_IPTsc_high,cost_IPTsc_low,cost_IPTsc_mean
232,6,450.495,3704.0700,5455.9950,2202.420,3829.20750
233,7,452.559,11163.1220,16442.9770,5732.414,11087.69550
234,8,448.632,3688.7520,14356.2240,11116.104,12736.16400
235,9,450.663,11116.3540,43263.6480,32597.957,37930.80250
256,30,0.000,6920.4225,8039.1975,1118.775,4578.98625
257,31,0.000,13062.9605,15174.7555,2111.795,8643.27525
258,32,0.000,6858.7200,13638.2400,6779.520,10208.88000
259,33,0.000,13032.6505,25914.8085,12882.158,19398.48325


In [1]:
plt.figure(dpi=300)
# old = bar["{}_incidence{}_BASELINE".format(case_type, agebin)]
# new = bar["{}_incidence{}".format(case_type, agebin)]
cases_averted = 5000*(df_itn["clinical_incidence_all_BASELINE"]-df_itn["clinical_incidence_all"])
cost = df_itn["cost_ITNs"]-np.mean(df_plot["cost_ITNs"])
frac_cases_averted = df_itn["frac_averted_clinical__all"]
plt.scatter(frac_cases_averted, cost/cases_averted, label="More ITN coverage", color='black', marker='D')

for i, sdf in df_plot.groupby("scenario_number"):
    label = sdf["comments"].iloc[0]
    cases_averted = 5000*(sdf["clinical_incidence_all_BASELINE"]-sdf["clinical_incidence_all"])
    cost = 0.5*sdf["cost_IPTsc_mean"]
    frac_cases_averted = sdf["frac_averted_clinical__all"]
    # plt.scatter(cases_averted, cost/cases_averted)
    plt.errorbar(100*frac_cases_averted, cost/cases_averted, 
                 yerr=0.5*(sdf["cost_IPTsc_high"]-sdf["cost_IPTsc_mean"])/cases_averted, 
                 xerr=100*sdf["frac_averted_clinical__all_STDEV"],
                 label=label, marker=marker_dict[sdf["scenario_number"].iloc[0]],
                 fmt='o', capsize=3, elinewidth=1.2, errorevery=1, markersize=10)
    plt.xlabel("Percent of cases averted")
    plt.ylabel("Cost per case averted")
    
plt.legend(fontsize=8)
plt.xlim(left=0)
plt.ylim(bottom=0)
# plt.ylim([0,20])

plt.title("School-based vs mass campaign")
plt.tight_layout()

plt.savefig("school_vs_mass.pdf")
plt.savefig("school_vs_mass.png")


NameError: name 'plt' is not defined